### Introduction to Tensorflow

In [1]:
import tensorflow as tf

# constant nodes -> they don't have an input - example : a bias
node1 = tf.constant(3.0, dtype=tf.float32, name="node1")
node2 = tf.constant(4.0, name="blablahhh") # also tf.float32 implicitly
node3 = tf.constant(5.0)

# this will print the nodes' characteristics not the constants
print(node1, node2, node3)

Tensor("node1:0", shape=(), dtype=float32) Tensor("blablahhh:0", shape=(), dtype=float32) Tensor("Const:0", shape=(), dtype=float32)


In [2]:
# let's start a session and evaluate the computational graph :
sess = tf.Session()
print(sess.run([node1, node2, node3])) # will print the node's outputs

[3.0, 4.0, 5.0]


We can build more complicated computations by combining Tensor nodes with operations (Operations are also nodes). 
For example, we can add our two constant nodes and produce a new graph as follows:

In [3]:
from __future__ import print_function
node3 = tf.add(node1, node2)
print("node3:", node3)
print("sess.run(node3):", sess.run(node3))

node3: Tensor("Add:0", shape=(), dtype=float32)
sess.run(node3): 7.0


As it stands, this graph is not especially interesting because it always produces a constant result. A graph can be parameterized to accept external inputs, known as placeholders. A placeholder is a promise to provide a value later.

In [4]:
a = tf.placeholder(tf.float32)
b = tf.placeholder(tf.float32)
adder_node = a + b  # + provides a shortcut for tf.add(a, b)

The preceding three lines are a bit like a function or a lambda in which we define two input parameters (a and b) and then an operation on them. We can evaluate this graph with multiple inputs by using the feed_dict argument to the run method to feed concrete values to the placeholders:

In [5]:
print(sess.run(adder_node, {a: 3, b: 4.5}))
print(sess.run(adder_node, {a: [1, 3], b: [2, 4]}))

7.5
[ 3.  7.]


We can make the computational graph more complex by adding another operation. For example,

In [6]:
add_and_triple = adder_node * 3.
print(sess.run(add_and_triple, {a: 3, b: 4.5}))

22.5


In machine learning we will typically want a model that can take arbitrary inputs, such as the one above. To make the model trainable, we need to be able to modify the graph to get new outputs with the same input. Variables allow us to add trainable parameters to a graph. They are constructed with a type and initial value:

In [7]:
W = tf.Variable([.3], dtype=tf.float32) # Variables can be initialized - not to be confused with constants though -
b = tf.Variable([-.3], dtype=tf.float32)
x = tf.placeholder(tf.float32)
linear_model = W*x + b

Constants are initialized when you call tf.constant, and their value can never change. By contrast, variables are not initialized when you call tf.Variable. To initialize all the variables in a TensorFlow program, you must explicitly call a special operation as follows:

In [19]:
init = tf.global_variables_initializer() # this was not needed before, since we did not have constants but variables -
sess.run(init) # initializes the variables

init is a handle to the TensorFlow sub-graph that initializes all the global variables. Until we call sess.run, the variables are uninitialized.

Since x is a placeholder, we can evaluate linear_model for **several values** of x simultaneously as follows:

In [20]:
print(sess.run(linear_model, {x: [1, 2, 3, 4]}))

[ 0.          0.30000001  0.60000002  0.90000004]


We've created a model, but we don't know how good it is yet. To evaluate the model on training data, we need a y placeholder to provide the desired values, and we need to write a loss function.

A loss function measures how far apart the current model is from the provided data. We'll use a standard loss model for linear regression, which sums the squares of the deltas between the current model and the provided data. linear_model - y creates a vector where each element is the corresponding example's error delta. We call tf.square to square that error. Then, we sum all the squared errors to create a single scalar that abstracts the error of all examples using tf.reduce_sum:

In [21]:
y = tf.placeholder(tf.float32)
squared_deltas = tf.square(linear_model - y)
loss = tf.reduce_sum(squared_deltas)
print(sess.run(loss, {x: [1, 2, 3, 4], y: [0, -1, -2, -3]}))

# if y matches exactly the results of linear_model calculated above - then loss function is at 0
print(sess.run(loss, {x: [1, 2, 3, 4], y: [0, 0.3, 0.6, 0.90000004]}))


23.66
0.0


We could improve this manually by reassigning the values of W and b to the perfect values of -1 and 1. A variable is initialized to the value provided to tf.Variable but can be changed using operations like tf.assign. For example, W=-1 and b=1 are the optimal parameters for our model. We can change W and b accordingly:

In [23]:
fixW = tf.assign(W, [-1.])
fixb = tf.assign(b, [1.])
sess.run([fixW, fixb]) # initializes the fixW and fixb variables
print(sess.run(loss, {x: [1, 2, 3, 4], y: [0, -1, -2, -3]})) # 0 - boom !

0.0
